Import libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner
!pip3 install lxml
!pip install BeautifulSoup4
!pip install geopy

from bs4 import BeautifulSoup

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library




print('Libraries imported.')

Scrape webpage and generate dataframe

In [ ]:
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'html.parser')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

Discard rows with Borough values 'Not assigned'

In [ ]:
df = df[df.Borough != 'Not assigned']
df.head()

In [ ]:
df.shape

Load the coordinates

In [ ]:
# Storing the address in 'url' variable
url = "http://cocl.us/Geospatial_data"
# Importing the file from the address contained in 'url' into 'df' 
coordinates = pd.read_csv(url)
coordinates.head()

Join both dataframes on Postal Code, in order to add coordinates to the main dataframe:

In [ ]:
df=df.join(coordinates.set_index('Postal Code'), on='PostalCode')
df.head()

Select just the Toronto's borough

In [ ]:
toronto_data=df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

Drawing the map

In [20]:
# create map of Toronto using latitude and longitude values
latitude=43.651070
longitude=-79.347015
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto